#### Perform structural shifts of the datasets
This file assumes you got the conllu files for your dataset.

In [51]:
import os
from stanza.utils.conll import CoNLL
from tqdm import tqdm
from datasets import DatasetDict
from datasets import Dataset
from datasets import load_dataset
from datasets import list_datasets

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def dict2text(stanza_sentence_dict):
    tokens = []
    for token in stanza_sentence_dict:
        tokens += [token["text"]]
    return " ".join(tokens)

You can split bigger dataset into smaller chunks so that they fit in your memory.

In [ ]:
# Get the training, and split the file, and galactic change.
filename = "../../data-files/wikitext-15M-conllu/wikitext-15M-train.conllu"
with open(filename) as f:
    content = f.readlines()
    
count = 0
current_file = []
partition_file = []
for c in content:
    current_file += [c]
    if len(c.strip()) == 0:
        partition_file += [current_file]
        current_file = []
        count += 1
assert count == len(partition_file)

NUM_PARTITION = 10
partition_file = [c for c in chunks(partition_file, len(partition_file)//NUM_PARTITION)]
output_dir = "../../data-files/wikitext-15M-conllu"
basename  = "wikitext-15M-train"
file_counter = 0
for file in partition_file:
    # each file is a list of doc
    output_file = f"{output_dir}/{basename}-partition-{file_counter}.conllu"
    with open(output_file, 'a') as the_file:
        for f in file:
            for l in f:
                the_file.write(l)
    print(f"write to {output_file} with doc_number={len(file)}")
    file_counter += 1

You will need to call following java scripts to perform structural changes.

In [ ]:
# We will run the following line to convert our conllu file using JAVA.
# types of shifts:
# wiki-text (galactic-en~fr@N~fr@V)
# wiki-text (galactic-en~ja_ktc@N~ja_ktc@V)
# wiki-text (galactic-en~fr@N~ja_ktc@V)

In [ ]:
cmd = f"GALACTIC_ROOT=../../submodules/gdtreebank/ \
        ../../submodules/gdtreebank/bin/gd-translate \
        --input ../../data-files/wikitext-15M-conllu/wikitext-15M-validation.conllu \
        --spec en~ja_ktc@N~ja_ktc@V"
print(f"starting command")
os.system(cmd)

In [ ]:
cmd = f"GALACTIC_ROOT=../../submodules/gdtreebank/ \
        ../../submodules/gdtreebank/bin/gd-translate \
        --input ../../data-files/wikitext-15M-conllu/wikitext-15M-test.conllu \
        --spec en~ja_ktc@N~ja_ktc@V"
print(f"starting command")
os.system(cmd)

In [ ]:
# run java program over smaller files.
for i in range(NUM_PARTITION+1):
    cmd = f"GALACTIC_ROOT=../../submodules/gdtreebank/ \
            ../../submodules/gdtreebank/bin/gd-translate \
            --input ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-{i}.conllu \
            --spec en~fr@N~ja_ktc@V"
    print(f"starting command-{i}")
    os.system(cmd)

# --input ../../data-files/wikitext-15M-conllu/wikitext-15M-train.conllu \
# --input ../../data-files/wikitext-15M-conllu/wikitext-15M-test.conllu \
# --input ../../data-files/wikitext-15M-conllu/wikitext-15M-validation.conllu \

Now, we need to combine and transfer conllu files into dataset.

In [95]:
# first, let us combine all sub-chunks together.
all_content = []
condition = "en~fr@N~ja_ktc@V"
for i in range(NUM_PARTITION+1):
    subfile_name = f"../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-{i}-{condition}.conllu"
    with open(subfile_name) as f:
        content = f.readlines()
    all_content.extend(content)
    print(f"processing: {subfile_name}")

processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-0-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-1-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-2-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-3-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-4-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-5-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-6-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-7-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-8-en~fr@N~ja_ktc@V.conllu
processing: ../../data-files/wikitext-15M-conllu/wikitext-15M-tr

In [96]:
jumbo_file = f"../../data-files/wikitext-15M-conllu/wikitext-15M-train-{condition}.conllu"
print(f"writing all combined files to: {jumbo_file}")
with open(jumbo_file, 'a') as the_file:
    for l in all_content:
        the_file.write(l)

writing all combined files to: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-en~fr@N~ja_ktc@V.conllu


In [97]:
# removing all files.
for i in range(NUM_PARTITION+1):
    subfile_name = f"../../data-files/wikitext-15M-conllu/wikitext-15M-train-partition-{i}-{condition}.conllu"
    os.remove(subfile_name)

We then need to load all conllu files and turn them into wiki-text data format.

In [62]:
condition = "en~fr@N~ja_ktc@V"

In [63]:
conllu_filename = f"../../data-files/wikitext-15M-conllu/wikitext-15M-train-{condition}.conllu"
print(f"reading in conllu file: {conllu_filename}...")
to_sent_doc = CoNLL.conll2doc(conllu_filename)
texts = []
for i in tqdm(range(len(to_sent_doc.sentences))):
    text = dict2text(to_sent_doc.sentences[i].to_dict())
    texts += [text]
train_dataset = {
    "text": texts
}
train_dataset = Dataset.from_dict(train_dataset)

conllu_filename = f"../../data-files/wikitext-15M-conllu/wikitext-15M-validation-{condition}.conllu"
print(f"reading in conllu file: {conllu_filename}...")
to_sent_doc = CoNLL.conll2doc(conllu_filename)
texts = []
for i in tqdm(range(len(to_sent_doc.sentences))):
    text = dict2text(to_sent_doc.sentences[i].to_dict())
    texts += [text]
validation_dataset = {
    "text": texts
}
validation_dataset = Dataset.from_dict(validation_dataset)

conllu_filename = f"../../data-files/wikitext-15M-conllu/wikitext-15M-test-{condition}.conllu"
print(f"reading in conllu file: {conllu_filename}...")
to_sent_doc = CoNLL.conll2doc(conllu_filename)
texts = []
for i in tqdm(range(len(to_sent_doc.sentences))):
    text = dict2text(to_sent_doc.sentences[i].to_dict())
    texts += [text]
test_dataset = {
    "text": texts
}
test_dataset = Dataset.from_dict(test_dataset)

reading in conllu file: ../../data-files/wikitext-15M-conllu/wikitext-15M-train-en~fr@N~ja_ktc@V.conllu...


100%|██████████| 174738/174738 [01:48<00:00, 1613.63it/s]


reading in conllu file: ../../data-files/wikitext-15M-conllu/wikitext-15M-validation-en~fr@N~ja_ktc@V.conllu...


100%|██████████| 2461/2461 [00:01<00:00, 1588.45it/s]


reading in conllu file: ../../data-files/wikitext-15M-conllu/wikitext-15M-test-en~fr@N~ja_ktc@V.conllu...


100%|██████████| 2891/2891 [00:01<00:00, 1786.44it/s]


In [64]:
# save it as dataset format.
datasets = DatasetDict({"train":train_dataset, 
                        "validation":validation_dataset, 
                        "test":test_dataset})
datasets.save_to_disk(f"../../data-files/wikitext-15M-{condition}")